In [ ]:
# Bare Minimum - One Sheet Script

In [ ]:
import pandas as pd
import numpy as np
import random
from random import randrange
import time
import os
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import itertools
from itertools import permutations
from itertools import combinations

In [ ]:
# Get all permutations of [1, 2, 3] 
perm = permutations([1, 2, 3]) 
 
# Print the obtained permutations 
for i in list(perm): 
    print (i) 

## Setup

In [ ]:
# Environment
class world:
    def __init__(self,Size):
        self.Size = Size
        self.Fig = None
        self.Grid = None
        self.Coin_Locations = []

def Generate_Map(world):
    # generate a grid with the dimensions of world.Size
    grid = np.zeros([world.Size,world.Size])
    
    fig = plt.figure(figsize = [5,5])
    plt.xlim(0,world.Size)
    plt.ylim(0,world.Size)

    world.Grid = grid
    world.Fig = fig


def Generate_Coins(world,number_of_coins):

    for coin in range(0,number_of_coins):
        coin_location_x = randrange(0,world.Size)
        coin_location_y = randrange(0,world.Size)

        coin_location = [coin_location_x,coin_location_y]
        world.Coin_Locations.append(coin_location)

    fig = world.Fig

    for coins in range(len(world.Coin_Locations)):
        world.Grid[world.Coin_Locations[coins][0],world.Coin_Locations[coins][1]] = 1
    
    plt.imshow(world.Grid, cmap='viridis', interpolation='nearest')
    plt.show()

    


In [ ]:
test_world = world(16)
Generate_Map(test_world)
Generate_Coins(test_world,32)

In [ ]:
class entity:
    def __init__(self,world):
        self.world = world
        self.IsSpawned = False

        if self.IsSpawned == False:
            self.Location = [0,0]
        elif self.IsSpawned == True:
            x_loc = world.Grid[np.where(world.Grid[:,0] == max(world.Grid))]
            print('x: ',x_loc)
            y_loc = world.Grid[np.where(world.Grid[0,:] == max(world.Grid))]
            print('y: ',y_loc)
            self.Location = [x_loc,y_loc]

        self.Speed = 6
        self.Coins = 0


def Spawn(entity):
    world = entity.world
    grid = world.Grid

    grid[8,8] = 2
    
    entity.IsSpawned = True
    entity.Location = [8,8]


    plt.imshow(grid, cmap='viridis', interpolation='nearest')
    plt.xlim(-1,world.Size)
    plt.ylim(-1,world.Size)
    plt.grid()
    plt.show()

In [ ]:
# Actions
## Search
def Search(Actor,world):
    actor_x = Actor.X
    actor_y = Actor.Y

    distances_to_coins = []
    for coin in range(len(world.Coin_Locations)):
        coin_location = world.Coin_Locations[coin]

        distance = 0
        distances_to_coins.append(distance)
    shortest_distance = distances_to_coins.sort()
    return shortest_distance[0]


## Dash
def Dash(Actor):
    pass

## Object Interaction
def Object_Interaction(Actor):
    locations = Actor.world.Grid
    rows, cols = locations.shape
        
    # Find the location of the agent (value 2)
    agent_location = Actor.Location

    #print(agent_location)
    #print('')
    
    spaces_within_reach = []

    # Iterate over the nearby locations within ±1 distance
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            x = agent_location[0] + dx
            y = agent_location[1] + dy
            
            # Check if the location is within bounds and is not the agent's location itself
            if (0 <= x < rows) and (0 <= y < cols) and (dx != 0 or dy != 0):
                spaces_within_reach.append([x, y])
    
    # now if only 1 of the spaces in spaces_within_reach match those from the world.Gold_Location_X or Y, then increase entity.coins by 1
    spaces_with_coins = []
    coin_locations = Actor.world.Coin_Locations
    for space in spaces_within_reach:
        for location in coin_locations:
            if space == location: 
                spaces_with_coins.append(space)
    
    if len(spaces_with_coins) == 1:
        space = spaces_with_coins[0]
        Actor.Coins += 1
        Actor.world.Grid[space[0],space[1]] = 0

    elif len(spaces_with_coins) > 1:
        space_choice = random.choice(spaces_with_coins)
        Actor.Coins += 1
        Actor.world.Grid[space_choice[0],space_choice[1]] = 0
    
    else: 
        print('no coins in reach')

    plt.imshow(Actor.world.Grid, cmap='viridis', interpolation='nearest')
    plt.xlim(-1,Actor.world.Size)
    plt.ylim(-1,Actor.world.Size)
    plt.grid()
    plt.show()


In [ ]:
def End_Turn(Actor):
    Actor.Speed = 6

In [ ]:
# Choose to Move
def Choose_Move(Actor):
    locations = Actor.world.Grid
    rows, cols = locations.shape
    moves_remaining = Actor.Speed
    agent_location = Actor.Location


    spaces_within_movement = []
    for dx in range(-5, 6):
        for dy in range(-5, 6):
            x = agent_location[0] + dx
            y = agent_location[1] + dy

            # Check if the location is within bounds and is not the agent's location itself
            if (0 <= x < rows) and (0 <= y < cols) and (dx != 0 or dy != 0):
                spaces_within_movement.append([x, y])

    # need to go through and rank each space on how close it is to a coin
    spaces_ranked = []
    for space in spaces_within_movement:
        # rank them on distance to coin, then sort by lowest to highest
        distances_to_coins = []
        for coin in Actor.world.Coin_Locations:
            distance_to_coin = math.dist(space,coin)
            distances_to_coins.append(distance_to_coin)
        distances_to_coins.sort(reverse=True)

        distance_to_nearest_coin = distances_to_coins[0]

        space_info = [space[0], space[1], distance_to_nearest_coin]
        spaces_ranked.append(space_info)

    # sort the spaces_ranked list by the third value in lists
    spaces_ranked.sort(key=lambda x: x[2])

    def create_heatmap(Actor, spaces_ranked):
        # Extract x, y coordinates and values
        x_coords = [space[0] for space in spaces_ranked]
        y_coords = [space[1] for space in spaces_ranked]
        values = [space[2] for space in spaces_ranked]

        # Determine the size of the grid
        x_max = max(x_coords) + 1
        y_max = max(y_coords) + 1

        # Create a 2D numpy array filled with NaN
        heatmap_data = np.full((x_max, y_max), np.nan)

        # Fill in the values
        for x, y, value in spaces_ranked:
            heatmap_data[x, y] = value

        # Create the heatmap
        plt.figure(figsize = [5,5])
        plt.imshow(heatmap_data, cmap='hot_r', interpolation='nearest')
        plt.xlim(0,Actor.world.Size)
        plt.ylim(0,Actor.world.Size)


        # Add text annotations
        #for i in range(heatmap_data.shape[0]):
        #    for j in range(heatmap_data.shape[1]):
        #        if not np.isnan(heatmap_data[i, j]):
        #            plt.text(j, i, f'{heatmap_data[i, j]:.1f}', 
        #                    ha='center', va='center', color='white')

        plt.tight_layout()
        plt.show()
    
    create_heatmap(Actor, spaces_ranked)
    ideal_location = spaces_ranked[0]
    if agent_location[0:1] == ideal_location[0:1]:
        return agent_location[0], agent_location[1]
    else:
        return ideal_location[0], ideal_location[1]

def Move_One(Actor):
    locations = Actor.world.Grid
    rows, cols = locations.shape
    moves_remaining = Actor.Speed
    agent_location = Actor.Location
    
    adjacent_spaces = []
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            x = agent_location[0] + dx
            y = agent_location[1] + dy

            # Check if the location is within bounds and is not the agent's location itself
            if (0 <= x < rows) and (0 <= y < cols) and (dx != 0 or dy != 0):
                adjacent_spaces.append([x, y])
    
    # choose a location to move into

    ideal_location = Choose_Move(Actor)
    # identify which adjacent_spot gets the agent closest to ideal location and return space_to_move_to
    space_to_move_to = min(adjacent_spaces, key=lambda space: math.dist(space, ideal_location))

    # The rest of the function continues here...
    space_current_occupancy = locations[space_to_move_to[0], space_to_move_to[1]]
    
    if space_current_occupancy == 1:
        Actor.world.Grid[space_to_move_to[0],space_to_move_to[1]] = 3
    #  elif Actor.world.Grid[space_to_move_to[0],space_to_move_to[1]] == 3:
    else:
        Actor.world.Grid[space_to_move_to[0],space_to_move_to[1]] = 2

    if Actor.world.Grid[agent_location[0],agent_location[1]] == 2:
        Actor.world.Grid[agent_location[0],agent_location[1]] = 0

    elif Actor.world.Grid[agent_location[0],agent_location[1]] == 3:
        Actor.world.Grid[agent_location[0],agent_location[1]] = 1
    
    grid = Actor.world.Grid

    Actor.Location = space_to_move_to

    plt.imshow(grid, cmap='viridis', interpolation='nearest')
    plt.xlim(-1,Actor.world.Size)
    plt.ylim(-1,Actor.world.Size)
    plt.grid()
    plt.show()


def Move_Full(Actor):
    pass


def Choose_Turn(Actor):
    subaction_dict = {
        '0': 'Move_One(Actor)',
        '1': 'Move_One(Actor)',
        '2': 'Move_One(Actor)',
        '3': 'Move_One(Actor)',
        '4': 'Move_One(Actor)',
        '5': 'Move_One(Actor)',
        '6': 'Object_Interaction(Actor)',
        '7': 'Dash(Actor)',
        '8': 'Object_Interaction(Actor)',
    }


    # now to convert all_options into all_options_numbers
    all_option_numbers = []
    for option in range(len(list(subaction_dict))):
        all_option_numbers.append(option)
        
    #print(all_option_numbers)

    # what does this function need to return?
    # it needs to return the order in which the sub-actions are taken (subactions being action, object interact, and each move step)
    
    # how should it return that in a way that is iterable/runnable?
    # the outcome will probably be returned as a list
    # that list will be a list of numbers, and the numbers will be an index for a second list that contains functions which will get called like piano keys
    # then the take_turn function will simply iterate through the list calling those functions in order
    
    # how to determine the order in which subactions should be taken?
    # the ideal subaction order will maximize the number of coins picked up
    # so I'll need to calculate a theoretical number of how many coins can be picked up per subaction permutation, sort, and choose the best

    # what type of rules are needed to determine the generation of the options?
    # 1. Moving can happen at any time but no more than 6 times total
    # 2. Object interaction can happen at any time but only once except if used as an action
    # 3. Action can only happen once and can be the Dash action or the Object Interaction action

    def is_valid_turn(turn):
        move_count = sum(1 for action in turn if action in range(6))
        object_interact_count = sum(1 for action in turn if action in [6, 8])
        action_count = sum(1 for action in turn if action in [7, 8])

        return (move_count <= 6 and 
                object_interact_count <= (2 if 8 in turn else 1) and 
                action_count <= 1)

    def generate_valid_turns():
        for length in range(1, 9):  # Turns can be 1 to 8 actions long
            for turn in itertools.product(all_option_numbers, repeat=length):
                if is_valid_turn(turn):
                    yield turn

    def evaluate_turn(turn):
        # This is a placeholder function. You'll need to implement the logic
        # to calculate how many coins can be picked up for a given turn.
        # This might involve simulating the turn on a copy of the game state.
        return 0  # Replace with actual evaluation

    valid_turns = list(generate_valid_turns())
    #print(valid_turns)
    best_turn = max(valid_turns, key=evaluate_turn)

    return [subaction_dict[str(action)] for action in best_turn]


def Action(Actor):
    pass

# Take_Turn
def Take_Turn(Actor):
    Action(Actor)
    Object_Interaction(Actor)



    End_Turn(Actor)
    

In [ ]:
commoner = entity(test_world)
test_world.Grid
Spawn(commoner)
#Move_One(commoner)
#Object_Interaction(commoner)
Choose_Turn(commoner)


In [ ]:
commoner.Coins